<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b8f81ad7e9af9b88ce33d5f4915c55e24351600359d95742975db4552f93c5f9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 10:11:39
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -2.79 L (-1.94%)
Current PnL: -27.12 L (-17.52%)
CY Booked + Current PnL: -12.83 L (-8.29%)
-------------------
Total profit:  1.26 L
Total loss:  -28.39 L
-------------------
Total Booked + Current PnL: 14.15 L (11.07%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.75 L (65.0%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.28,-8.42,9.22,0.02,14739.0,-14698.0,159861.0,143.86,67.0,M-SC,15.52,234.0,-1.00,1.17,39.05,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.66,-17.26,20.97,0.09,17485.0,-17395.0,83382.0,93.60,35.0,M-SC,2.72,253.0,-0.99,0.61,9.54,OX40N,NTT,DURABLES
43,ITC,452.00,-0.00,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,45.0,X-LC,1.58,5.0,-0.13,1.45,3.96,X40,NTT,FMCG
50,MASFIN,398.61,-0.81,-2.80,25.57,22.05,24352.0,-2745.0,95235.0,-15.99,57.0,H-SC,6.97,164.0,-0.11,0.70,38.43,XR,ATH,FINANCE
35,ICICIGI,2252.93,0.44,5.69,13.73,20.19,25295.0,9913.0,184233.0,-17.20,47.0,X-MC,6.61,68.0,0.39,1.35,21.66,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,7.83,-42.78,204.34,74.15,191301.0,-69986.0,93619.0,-268.43,48.0,H-SC,19.89,149.0,-0.37,0.69,20.00,XR,NTT,CHEMICALS
24,EASEMYTRIP,26.40,2.44,-53.64,214.29,45.70,175647.0,-94848.0,81967.0,-14.90,64.0,M-SC,23.24,205.0,-0.54,0.60,18.14,XY24,NTT,TRAVEL
29,HAPPSTMNDS,1480.71,2.15,-36.60,191.56,84.85,155654.0,-46909.0,81256.0,-24.68,53.0,H-SC,14.03,147.0,-0.30,0.60,4.29,AR,ATH,IT
73,TCS,4389.97,2.06,-11.82,35.26,19.28,107570.0,-40879.0,305077.0,-22.98,71.0,X-LC,6.52,1.0,-0.38,2.24,12.78,X40,ATH,IT
40,INDUSINDBK,1800.00,1.66,-36.43,109.06,32.89,53523.0,-28130.0,49077.0,-706.04,64.0,L-MC,6.21,259.0,-0.53,0.36,35.15,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-80.17,-79.95,522.32,24.79,263787.0,-201353.0,50503.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
44,JCHAC,2282.0,-4.67,-34.59,52.86,-0.01,44192.0,-44203.0,83602.0,17008.70,8.0,M-SC,1.01,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
33,HINDUNILVR,2922.0,-4.23,-8.27,23.92,13.67,56403.0,-21263.0,235800.0,-15.11,34.0,X-LC,3.32,14.0,-0.38,1.73,11.32,XY25,NTT,FMCG
28,GREENPANEL,537.0,-3.90,-37.28,128.22,43.14,147230.0,-68252.0,114826.0,157.11,26.0,M-SC,7.20,240.0,-0.46,0.84,8.01,XY24,NTT,MISC
51,MEDANTA,1486.0,-3.83,-2.64,27.44,24.08,33915.0,-3354.0,123596.0,-10.46,33.0,X-SC,6.01,89.0,-0.10,0.91,16.46,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,-1.76,-0.73,49.72,48.63,98688.0,-1466.0,198488.0,-33.87,59.0,H-SC,14.75,136.0,-0.01,1.46,10.74,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.80,-6.80,123.08,107.91,169363.0,-10040.0,137604.0,-24.20,30.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.28,-8.42,9.22,0.02,14739.0,-14698.0,159861.0,143.86,67.0,M-SC,15.52,234.0,-1.00,1.17,39.05,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.66,-17.26,20.97,0.09,17485.0,-17395.0,83382.0,93.60,35.0,M-SC,2.72,253.0,-0.99,0.61,9.54,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.16,-24.03,49.36,13.47,101127.0,-64791.0,204876.0,-68.70,31.0,H-SC,2.39,158.0,-0.64,1.50,4.17,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,0.80,-24.83,75.63,32.03,74952.0,-32731.0,99104.0,-18.72,45.0,H-SC,7.88,148.0,-0.44,0.73,11.00,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.72,6.14,62.29,72.27,112034.0,10411.0,179859.0,-8.8,71.0,M-LC,2.96,99.0,0.09,1.32,12.7,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.72,6.14,62.29,72.27,112034.0,10411.0,179859.0,-8.80,71.0,M-LC,2.96,99.0,0.09,1.32,12.70,XR,NTT,IT
38,INDIAMART,4810.62,-1.25,-1.14,109.11,106.72,133033.0,-1410.0,121926.0,-51.01,36.0,H-SC,1.56,139.0,-0.01,0.90,21.97,AR,ATH,MISC
37,IEX,219.00,-1.76,-0.73,49.72,48.63,98688.0,-1466.0,198488.0,-33.87,59.0,H-SC,14.75,136.0,-0.01,1.46,10.74,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.52,-1.92,38.28,35.63,78685.0,-4022.0,205552.0,-15.85,40.0,H-MC,3.43,119.0,-0.05,1.51,14.43,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.80,-6.80,123.08,107.91,169363.0,-10040.0,137604.0,-24.20,30.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.17,-79.95,522.32,24.79,263787.0,-201353.0,50503.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.96,-21.45,88.98,48.45,140149.0,-43000.0,157506.0,-27.18,12.0,X-MC,9.43,64.0,-0.31,1.16,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.98,-38.20,118.78,35.20,94451.0,-49152.0,79518.0,4.26,22.0,X-SC,14.44,92.0,-0.52,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.05,-12.72,39.31,21.58,48080.0,-17829.0,122310.0,-56.03,24.0,X-MC,5.77,56.0,-0.37,0.90,8.28,X40N,ATH,IT
54,PGHH,17907.65,-0.31,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,28.0,X-MC,4.68,57.0,-0.15,1.39,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.32,-23.68,115.09,64.15,209003.0,-56351.0,181600.0,-55.44,41.0,X-SC,1.49,82.0,-0.27,1.33,1.87,XY24,BTT,CEMENT
8,AWL,485.0,0.77,-21.05,94.43,53.50,224956.0,-63526.0,238225.0,-61.84,31.0,X-MC,1.55,58.0,-0.28,1.75,5.81,XY24,NTT,FMCG
43,ITC,452.0,-0.00,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,45.0,X-LC,1.58,5.0,-0.13,1.45,3.96,X40,NTT,FMCG
15,CAMS,4762.0,-80.17,-79.95,522.32,24.79,263787.0,-201353.0,50503.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.62,1.96,28.44,30.97,51252.0,3472.0,180210.0,-22.06,50.0,X-MC,2.22,75.0,0.07,1.32,14.77,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.98,-38.20,118.78,35.20,94451.0,-49152.0,79518.0,4.26,22.0,X-SC,14.44,92.0,-0.52,0.58,0.0,X40,NTT,FOOTWEAR
15,CAMS,4762.00,-80.17,-79.95,522.32,24.79,263787.0,-201353.0,50503.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.0,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.96,-21.45,88.98,48.45,140149.0,-43000.0,157506.0,-27.18,12.0,X-MC,9.43,64.0,-0.31,1.16,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-2.27,-31.70,109.16,42.84,48463.0,-20610.0,44396.0,-55.37,34.0,X-SC,37.36,83.0,-0.43,0.33,0.0,XY24,NTT,MISC
54,PGHH,17907.65,-0.31,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,28.0,X-MC,4.68,57.0,-0.15,1.39,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.27,-31.70,109.16,42.84,48463.0,-20610.0,44396.0,-55.37,34.0,X-SC,37.36,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-80.17,-79.95,522.32,24.79,263787.0,-201353.0,50503.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
59,RELAXO,1176.00,-0.33,-47.57,190.51,52.31,144982.0,-69058.0,76102.0,-44.15,34.0,X-SC,5.38,91.0,-0.48,0.56,1.87,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.98,-38.20,118.78,35.20,94451.0,-49152.0,79518.0,4.26,22.0,X-SC,14.44,92.0,-0.52,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.05,-12.72,39.31,21.58,48080.0,-17829.0,122310.0,-56.03,24.0,X-MC,5.77,56.0,-0.37,0.90,8.28,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,2.06,-11.82,35.26,19.28,107570.0,-40879.0,305077.0,-22.98,71.0,X-LC,6.52,1.0,-0.38,2.24,12.78,X40,ATH,IT
41,INFY,2275.00,1.20,8.76,42.40,54.87,144283.0,27401.0,340289.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.50,17.77,X40,BTT,IT
75,TMPV,600.00,-0.52,-15.97,69.20,42.18,160726.0,-44147.0,232263.0,-24.30,31.0,X-LC,3.96,3.0,-0.27,1.71,0.61,XY24,NTT,AUTO
81,VBL,671.64,-0.31,-3.85,41.10,35.67,124816.0,-12154.0,303688.0,-15.30,57.0,X-LC,5.32,4.0,-0.10,2.23,9.47,X40N,ATH,FMCG
43,ITC,452.00,-0.00,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,45.0,X-LC,1.58,5.0,-0.13,1.45,3.96,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.66,-36.43,109.06,32.89,53523.0,-28130.0,49077.0,-706.04,64.0,L-MC,6.21,259.0,-0.53,0.36,35.15,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.28,-17.01,118.92,81.67,92579.0,-15960.0,77850.0,6994.44,38.0,L-SC,16.28,271.0,-0.17,0.57,50.76,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.81,-2.80,25.57,22.05,24352.0,-2745.0,95235.0,-15.99,57.0,H-SC,6.97,164.0,-0.11,0.70,38.43,XR,ATH,FINANCE
13,BSOFT,831.70,-0.09,-21.05,90.98,50.79,101035.0,-29601.0,111052.0,1.28,75.0,H-SC,5.20,151.0,-0.29,0.82,29.30,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.28,-8.42,9.22,0.02,14739.0,-14698.0,159861.0,143.86,67.0,M-SC,15.52,234.0,-1.00,1.17,39.05,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.28,-8.42,9.22,0.02,14739.0,-14698.0,159861.0,143.86,67.0,M-SC,15.52,234.0,-1.00,1.17,39.05,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.17,3.19,17.21,20.95,44711.0,8026.0,259794.0,2.18,78.0,X-LC,18.45,31.0,0.18,1.91,39.96,X40,ATH,PAINTS
13,BSOFT,831.70,-0.09,-21.05,90.98,50.79,101035.0,-29601.0,111052.0,1.28,75.0,H-SC,5.20,151.0,-0.29,0.82,29.30,XR,ATH,IT
41,INFY,2275.00,1.20,8.76,42.40,54.87,144283.0,27401.0,340289.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.50,17.77,X40,BTT,IT
32,HCLTECH,1922.01,1.63,8.20,15.28,24.73,39998.0,19842.0,261766.0,13.18,69.0,X-LC,5.23,7.0,0.50,1.92,24.69,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.62
2,50,76.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.25
MC    30.03
LC    24.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      22.38
X40N     10.81
XR       10.13
AR        9.43
XY25      9.32
OX40N     7.78
SR        1.03
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.68
X-MC    23.32
X-LC    21.17
M-SC    12.22
X-SC     6.95
H-MC     4.94
M-MC     1.41
L-SC     1.40
M-LC     1.32
H-LC     1.23
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.33,-6.69,41.20
IT,12.79,-15.78,78.37
FINANCE,9.61,-18.52,68.60
ELECTRICAL,6.27,-10.06,50.38
PAINTS,6.22,-8.85,25.02
MISC,6.16,-55.11,118.65
INSURANCE,4.53,-2.33,38.39
PHARMA,4.08,-0.71,32.94
AUTO,3.45,-22.35,74.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3174983.0,21
AR,1289789.0,10
XR,1284516.0,13
X40,991994.0,14
X40N,976003.0,9
OX40N,730186.0,10
XY25,372689.0,6
SR,280213.0,2
MH,75059.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3619528.0,25
M-SC,1393184.0,15
X-MC,1306929.0,16
X-SC,973846.0,8
X-LC,881212.0,11
H-MC,407043.0,3
L-SC,269277.0,3
M-LC,112034.0,1
H-LC,66235.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1271455.0      6
           AR         886626.0      5
M-SC       XY24       794703.0      6
H-SC       XR         768248.0      7
X-SC       X40N       503177.0      3
X-MC       X40        452217.0      7
           XY24       388969.0      3
X-LC       X40        360489.0      5
H-SC       OX40N      337927.0      4
M-SC       OX40N      304552.0      5
X-SC       XY24       291381.0      3
H-SC       SR         280213.0      2
X-MC       X40N       274531.0      4
X-LC       XY24       233572.0      2
H-MC       AR         212140.0      2
X-LC       X40N       198295.0      2
H-MC       XY24       194903.0      1
X-MC       XY25       191212.0      2
L-SC       XR         181570.0      2
X-SC       X40        179288.0      2
M-SC       XR         169141.0      2
           AR         124788.0      2
M-LC       XR         112034.0      1
X-LC       XY25        88856.0      2
L-SC       OX40N       87707.0      1
H-SC       MH          75059.0      1
H-LC       AR          66235.0      1
L-MC       XR          53523.0      1
M-MC       XY25        53042.0      1
L-LC       XY25        39579.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
